In [6]:
import numpy
import h5py
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [7]:
#Load data and process it

filename = "country_lyrics_all.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()[:100000]

#list of characters
chars = sorted(list(set(raw_text)))

#create bigram vocabulary
bigrams = []
for i in chars: 
    for j in chars: 
        bigrams.append(str(i) + str(j))
        
# create mapping of unique bigrams to integers, and a reverse mapping
bigram_to_int = dict((c, i) for i, c in enumerate(bigrams))
int_to_bigram = dict((i, c) for i, c in enumerate(bigrams))

# summarize the loaded data
n_bigrams = int(len(raw_text) / 2)
bigram_vocab = len(bigrams)

print("Total Characters: ", n_bigrams)
print("Total vocab: ", bigram_vocab)
print(len(bigram_to_int), len(int_to_bigram))

Total Characters:  50000
Total vocab:  2304
2304 2304


In [8]:
seq_length = 50 # can be changed later
bigram_size = 2
dataX = []
dataY = []
#iterate through 
for i in range(0, n_bigrams - seq_length * bigram_size, 2):
    #split training data into dataset
    seq_in = raw_text[i: i + seq_length * bigram_size]
    seq_out = raw_text[i + seq_length:i + seq_length + 2]
    
    #for loop to create X data
    temp_X = []
    for i in range(0, seq_length * bigram_size, 2): 
        bigram = seq_in[i:i+2]
        temp_X.append(bigram_to_int[bigram])
    dataX.append(temp_X)
    #get y character
    dataY.append(bigram_to_int[seq_out])
    
    if i % 100000 == 0: 
        print('here')

n_patterns = len(dataX)

print(len(dataX), len(dataX[0]))
print ("Total Patterns: ", len(dataX))

24950 50
Total Patterns:  24950


In [9]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(bigram_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print(len(y))

24950


In [10]:
print(X.shape) #(99900, 100, 1)
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(X, y, epochs=1, batch_size=1024) #error: Error when checking target: expected dense_6 to have 3 dimensions, but got array with shape (99900, 48)

print("Done!!!")

(24950, 50, 1)
Epoch 1/1
24950/24950 [==============================] - 336s - loss: 5.8076   
Done!!!


In [ ]:
# weights = model.get_weights()
# numpy.save("jank_weights", weights)
# updated_weights = numpy.load("jank_weights.npy")
# model.set_weights(updated_weights)
# model.fit(X, y, epochs=1, batch_size=128) 


In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]

print ("Seed:")
print ("\"", ''.join([int_to_bigram[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))

    x = x / float(bigram_vocab)
    prediction = model.predict(x, verbose=0)
    #print(len(prediction[0]))
    
    # index = numpy.argmax(prediction)
    index = numpy.random.choice(numpy.arange(0, len(prediction[0])), p=prediction[0])
    result = int_to_bigram[index]
    seq_in = [int_to_bigram[value] for value in pattern]

    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")